Construcción de una CNN usando estimadores
===

* *30 min* | Última modificación: Julio 12, 2019.

## Preparación

In [1]:
import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.ERROR)

## Modelo

In [4]:
def cnn_model_fn(features, labels, mode):
    """Model function for CNN."""
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, 
        pool_size=[2, 2], 
        strides=2)

    # Convolutional Layer #2 and Pooling Layer #2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(
        inputs=conv2, 
        pool_size=[2, 2], 
        strides=2)

    # Dense Layer
    pool2_flat = tf.reshape(
        pool2, 
        [-1, 7 * 7 * 64])
    
    dense = tf.layers.dense(
        inputs=pool2_flat, 
        units=1024, 
        activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(
        inputs=dense, 
        rate=0.4, 
        training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits Layer
    logits = tf.layers.dense(
        inputs=dropout, 
        units=10)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(
        labels=labels, 
        logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            loss=loss, 
            train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels, 
            predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(
        mode=mode, 
        loss=loss, 
        eval_metric_ops=eval_metric_ops)

## Entrenamiento y evaluación

In [5]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

## Creación del estimador

In [6]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn, 
    model_dir="/tmp/mnist_convnet_model")

I0712 21:23:17.296927 140349469853504 estimator.py:1790] Using default config.
I0712 21:23:17.299389 140349469853504 estimator.py:209] Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa5585eb2b0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Hook

In [7]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

## Entrenamiento del modelo

In [8]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

# train one step and display the probabilties
mnist_classifier.train(
    input_fn=train_input_fn,
    steps=1,
    hooks=[logging_hook]);

W0712 21:24:26.058290 140349469853504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0712 21:24:26.077720 140349469853504 deprecation.py:323] From /home/vagrant/.local/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_queue_runner.py:62: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0712 21:24:26.080062 140349469853504 deprecation.py:323] From /home/vagrant/.local/lib/python3.6/site-packages/tensorflow_estimator/python/estimator/inputs/queues/feeding_fu

In [9]:
mnist_classifier.train(
    input_fn=train_input_fn, 
    steps=1000);

I0712 21:24:55.215213 140349469853504 estimator.py:1145] Calling model_fn.
I0712 21:24:55.468513 140349469853504 estimator.py:1147] Done calling model_fn.
I0712 21:24:55.471066 140349469853504 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0712 21:24:55.552362 140349469853504 monitored_session.py:240] Graph was finalized.
W0712 21:24:55.555237 140349469853504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
I0712 21:24:55.558950 140349469853504 saver.py:1280] Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1
W0712 21:24:55.585066 140349469853504 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1066: get_checkpoint_mtimes (from tensorflow.py

## Evaluación del modelo

In [10]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

I0712 21:26:58.943718 140349469853504 estimator.py:1145] Calling model_fn.
I0712 21:26:59.050471 140349469853504 estimator.py:1147] Done calling model_fn.
I0712 21:26:59.072779 140349469853504 evaluation.py:255] Starting evaluation at 2019-07-12T21:26:59Z
I0712 21:26:59.154744 140349469853504 monitored_session.py:240] Graph was finalized.
I0712 21:26:59.157262 140349469853504 saver.py:1280] Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-1001
I0712 21:26:59.195664 140349469853504 session_manager.py:500] Running local_init_op.
I0712 21:26:59.205178 140349469853504 session_manager.py:502] Done running local_init_op.
I0712 21:27:01.352412 140349469853504 evaluation.py:275] Finished evaluation at 2019-07-12-21:27:01
I0712 21:27:01.353321 140349469853504 estimator.py:2039] Saving dict for global step 1001: accuracy = 0.7254, global_step = 1001, loss = 1.8175361
I0712 21:27:01.385889 140349469853504 estimator.py:2099] Saving 'checkpoint_path' summary for global step 1001: /tmp/

{'accuracy': 0.7254, 'loss': 1.8175361, 'global_step': 1001}
